In [109]:
import pandas as pd
import numpy as np
import os

In [110]:
with open("sent_corpus.csv", "r") as sent_file:
    lines = sent_file.read().split("\n")

In [111]:
%time rows = [line.split(",") for line in lines if line]
%time rows = [row[:3] + [",".join(row[3:])] for row in rows]
# remove document start character 
rows[0][0] = rows[0][0][1:]

sentDf_cols = ['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText']


%time sentDf_a = pd.DataFrame(rows[1:],columns=sentDf_cols)

#print(sentDf)

sentDf_a[["ItemID","Sentiment"]] = sentDf_a[["ItemID","Sentiment"]].astype(int)
%time sentDf_a["SentimentText"] = sentDf_a["SentimentText"].apply(lambda text: text.split())

CPU times: user 3.25 s, sys: 1.89 s, total: 5.14 s
Wall time: 6.33 s
CPU times: user 4.48 s, sys: 157 ms, total: 4.64 s
Wall time: 4.65 s
CPU times: user 377 ms, sys: 52.4 ms, total: 429 ms
Wall time: 475 ms
CPU times: user 4.82 s, sys: 834 ms, total: 5.65 s
Wall time: 5.89 s


In [112]:
import gensim
%time w2vM = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
dim = 300



CPU times: user 1min 36s, sys: 11 s, total: 1min 47s
Wall time: 2min 7s


In [97]:
%time w2vM = gensim.models.Word2Vec(sentDf["SentimentText"])
dim = 100

CPU times: user 1min 21s, sys: 2.53 s, total: 1min 24s
Wall time: 38.5 s


In [113]:
path = os.getcwd()+"/fullData/"
with open(path+"us_train.labels", "r") as labels_file:
    lines = labels_file.read().split("\n")
    
labels = pd.DataFrame(lines[:], columns=['SentimentLabels'])
#print(labels)

In [114]:
path = os.getcwd()+"/fullData/"
with open(path+"us_train.text", "r") as tweet_text_file:
    lines = tweet_text_file.read().split("\n")
    
counter = 0
for line in lines:
    if(counter == 100):
        break
#    print(line)
    counter += 1
    
sentDf = pd.DataFrame(lines[:], columns=['SentimentText'])
%time sentDf["SentimentText"] = sentDf["SentimentText"].apply(lambda text: text.split())
#print(sentDf[1:])
    


CPU times: user 952 ms, sys: 28.4 ms, total: 981 ms
Wall time: 1.16 s


In [115]:
# number of samples to aggregate
Ns = int(1e5)

In [116]:
%%time
# MEAN AGGREGATION
tvecs = np.array([np.array([w2vM[t] if t in w2vM
                                else np.zeros((dim,))
                            for t in twt]).mean(axis=0)
                 for twt in sentDf['SentimentText'][:Ns]])

CPU times: user 10.4 s, sys: 1.97 s, total: 12.3 s
Wall time: 21 s


In [117]:
# number of samples to train on
N = int(1e5)
X = tvecs[:N]
y_pre = labels['SentimentLabels'][:N].values

y = np.array(y_pre).astype(np.int)

print(y)
print(X)

[ 2 17  0 ...,  9  9  6]
[[-0.03525391  0.06650391  0.06716309 ..., -0.05820313 -0.02873535
   0.03405762]
 [-0.00695801  0.06316584 -0.05490945 ..., -0.06491921  0.00937722
  -0.03728693]
 [-0.03508301 -0.05007324 -0.04908447 ..., -0.0609375  -0.00522461
   0.06296082]
 ..., 
 [ 0.02618408  0.02658081  0.0098877  ..., -0.05737305  0.02505493
   0.06640625]
 [-0.00427246  0.09061686  0.02342394 ..., -0.0339951   0.05254449
   0.02824571]
 [ 0.00064087  0.01641846  0.03424072 ..., -0.02520752  0.01345825
   0.00135803]]


In [118]:
# generate test/train split
ratio = 0.8
tidx = np.random.rand(N) < ratio
pidx = ~tidx

In [119]:
import sklearn
from sklearn import ensemble,svm,neural_network,discriminant_analysis
from sklearn.metrics import roc_curve,auc,precision_recall_curve

from matplotlib import pyplot as plt
% matplotlib inline

In [126]:
hidden_layer_config = (100,)

mlp = sklearn.neural_network.MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes= hidden_layer_config, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


%time mlp.fit(X[tidx],y[tidx])

#51% is baseline

CPU times: user 5min 59s, sys: 28.9 s, total: 6min 28s
Wall time: 4min 32s


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [127]:
y_pred_pre = mlp.predict(X[tidx])

y_pred = np.array(y_pred_pre).astype(np.int)

print(y_pred)

[ 2 17  0 ...,  0  1  1]


In [125]:
num_same = 0

for i in y:
    if(y_pred[i] == y[i]):
#        print("y pred: " + str(y_pred[i]) +" y: " + str(y[i]))
        num_same += 1
    
print(num_same)    
accuracy = float(num_same) / y.size
print(accuracy)

48138
0.48138


In [77]:
n_classes = 20


# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:i], y_pred[:i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

ValueError: Data is not binary and pos_label is not specified

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()